In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICE']="0"

In [2]:
import argparse 
from argparse import Namespace
import torch
import torchvision.transforms as transforms
import numpy as np
import itertools
import wandb

import sys

sys.path.append('../../')
import sopa.src.models.odenet_cifar10.layers as cifar10_models
from sopa.src.models.odenet_cifar10.utils import *
from sopa.src.models.odenet_cifar10.data import get_cifar10_test_loader
from sopa.src.solvers.utils import create_solver, noise_params, create_solver_ensemble_by_noising_params

import robustbench as rb 
import eagerpy as ep
import foolbox as fb
import MegaAdversarial.src.attacks as ma

In [3]:
data_root="./data"
test_loader = get_cifar10_test_loader(batch_size=32,
                                      data_root=data_root,
                                      num_workers=1,
                                      pin_memory=False,
                                      shuffle=False,
                                      download=True)
len(test_loader)

Files already downloaded and verified


312

# Source/target notation

We add prefix **'source'** to all model/solver options, if the **model/solver is used to generate adversarial images**.

We add prefix **'target'** to all model/solver options, if the **model/solver is used for the standard/robust accuracy evaluation**.

For example, when
- neural ODE attacks neural ODE: `source_model = target_model = 'premetanode10'`
- 'Wong2020Fast' from RobustBench attacks neural ODE: `source_model = 'Wong2020Fast', target_model = 'premetanode10'`


# Image preprocessing

We define three types of **image normalization**:

- `data_inv_normalize`: image normalization that converts data loader image range to [0, 1]

- `source_preprocessing `: image normalization that converts the image range from [0, 1] to the range of the source model's input.

- `target_normalize`: image normalization that converts the image range from [0, 1] to the range of the target model's input.

In [21]:
# Image normalization used by CIFAR10 data loader
cifar10_mean, cifar10_std = (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)

ROBUSTBENCH_NAMES = ['Wong2020Fast', 'Sehwag2021Proxy_R18', 'Carmon2019Unlabeled']

def get_input_shift_scale(model_name):
    input_shift = (0., 0., 0.) if model_name in ROBUSTBENCH_NAMES else cifar10_mean
    input_scale = (1., 1., 1.) if model_name in ROBUSTBENCH_NAMES else cifar10_std
    return input_shift, input_scale

#####  Define inverse transformation for images from CIFAR10 data loader: converts images to [0, 1] range.

In [6]:
dataset_shift = cifar10_mean if cifar10_mean is not None else (0., 0., 0.)
dataset_scale = cifar10_std if cifar10_std is not None else (1., 1., 1.)

data_inv_normalize = transforms.Normalize(mean=[-m/s for m, s in zip(dataset_shift, dataset_scale)],
                                          std=[1/s for s in dataset_scale])

# Example: CNN from RobustBench attacks neural ODE 

##### Get shifting and scaling factors that transform images from [0, 1] range to the range of source/target model inputs (i.e. inputs to the forward pass).

In [7]:
source_name, target_name = 'Wong2020Fast', 'premetanode10' 

source_input_shift, source_input_scale = get_input_shift_scale(source_name)
target_input_shift, target_input_scale = get_input_shift_scale(target_name)

##### Define preprocessing for FoolBox attack generator: converts images from [0, 1] range to the range of source model's inputs.

In [8]:
source_preprocessing = dict(mean=source_input_shift, std=source_input_scale, axis=-3)

##### Define image transformation before validation: converts images from [0, 1] range to the range of target model's inputs.

In [9]:
target_normalize = transforms.Normalize(mean=target_input_shift, std=target_input_scale)

## Solver/model configs

In [10]:
from black_box import get_solvers_solver_options_arr

dtype = torch.float32
device = 'cuda'

source_solvers_config =  Namespace(**{'solvers': None,
                                     'solver_mode': None,
                                     'switch_probs': None,
                                     'ensemble_prob': None,
                                     'ensemble_weights': None,
                                    })
source_model_ensemble_config = Namespace(**{'model_ensemble_size': 1,
                                            'average_mode': None,})



target_solvers_config = Namespace(**{'solvers': "rk2,u,8,-1,0.5,-1",
                                     'solver_mode': 'standalone',
                                     'switch_probs': None,
                                     'ensemble_prob': None,
                                     'ensemble_weights': None,
                                    })
target_model_ensemble_config = Namespace(**{'model_ensemble_size': 1,})

##### Load models

In [11]:
from black_box import init_model_metanode

source_model = rb.utils.load_model(model_name=source_name,
                                   threat_model='Linf',
                                   model_dir="./checkpoints_robustbench").eval()
source_model.to(device)



checkpoint_name = "./checkpoints/fgsm_random_8_255_smoothing_00125_seed_102_checkpoint_6125.pth"
checkpoint=torch.load(checkpoint_name)

target_model = init_model_metanode(checkpoint).eval()
target_model.to(device)
print()

##### Initialize source/target solvers

In [12]:
source_solvers_solver_options_arr = get_solvers_solver_options_arr(source_solvers_config,
                                                                   source_model_ensemble_config,
                                                                   dtype, device)
target_solvers_solver_options_arr = get_solvers_solver_options_arr(target_solvers_config,
                                                                   target_model_ensemble_config,
                                                                   dtype, device)

print(f'Solvers for source model: {source_solvers_solver_options_arr} \n' +
      f'Solvers for target model: {target_solvers_solver_options_arr}')

Solvers for source model: {} 
Solvers for target model: [{'solvers': [<sopa.src.solvers.rk_parametric_order2stage2.RKOrder2Stage2 object at 0x1554c323ab10>], 'solver_options': Namespace(ensemble_prob=None, ensemble_weights=None, solver_mode='standalone', switch_probs=None)}]


## Attack and evaluate

In [13]:
# FGSM attack
attack = fb.attacks.FGSM(random_start=False)

## PGD attack
# attack = fb.attacks.LinfPGD(steps=20,
#                             random_start=False,
#                             abs_stepsize=2/255.)

## DeepFool attack
# attack = fb.attacks.LinfDeepFoolAttack()

epsilons = [8/255.]

source_fmodel = fb.PyTorchModel(source_model, bounds=(0, 1), device=device, preprocessing=source_preprocessing)

In [14]:
from black_box import attack_validate

robust_accuracies = attack_validate(attack,
                                    epsilons,
                                    test_loader,
                                    source_model_ensemble_config,
                                    source_solvers_solver_options_arr,
                                    target_solvers_solver_options_arr,
                                    source_model=source_model,
                                    source_fmodel=source_fmodel,
                                    target_model=target_model,
                                    data_inv_normalize=data_inv_normalize,
                                    target_normalize=target_normalize,
                                    )

for eps, acc in zip(epsilons, robust_accuracies):
    print(f'eps: {int(eps*255)}, robust_accuracy: {acc}')

/beegfs/home/y.gusak/neural-ode-sopa/examples/cifar10/black_box.py:371: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  x = torch.tensor(clipped_advs[i].numpy()).to(device)


eps: 8, robust_accuracy: 0.585036039352417


# Example: neural ODE attacks neural ODE 

In [15]:
source_name, target_name = 'premetanode10', 'premetanode10' 

source_input_shift, source_input_scale = get_input_shift_scale(source_name)
target_input_shift, target_input_scale = get_input_shift_scale(target_name)

source_preprocessing = dict(mean=source_input_shift, std=source_input_scale, axis=-3)
target_normalize = transforms.Normalize(mean=target_input_shift, std=target_input_scale)

In [16]:
from black_box import get_solvers_solver_options_arr

dtype = torch.float32
device = 'cuda'

source_solvers_config =  Namespace(**{'solvers': "rk2,u,8,-1,0.5,-1",
                                     'solver_mode': 'standalone',
                                     'switch_probs': None,
                                     'ensemble_prob': None,
                                     'ensemble_weights': None,
                                    })
source_model_ensemble_config = Namespace(**{'model_ensemble_size': 1,
                                            'average_mode': None,})



target_solvers_config = Namespace(**{'solvers': "rk2,u,8,-1,0.5,-1",
                                     'solver_mode': 'standalone',
                                     'switch_probs': None,
                                     'ensemble_prob': None,
                                     'ensemble_weights': None,
                                    })
target_model_ensemble_config = Namespace(**{'model_ensemble_size': 1,})

In [17]:
from black_box import init_model_metanode

checkpoint_name = "./checkpoints/fgsm_random_8_255_smoothing_00125_seed_102_checkpoint_6125.pth"
checkpoint=torch.load(checkpoint_name)

source_model = init_model_metanode(checkpoint).eval()
source_model.to(device)


checkpoint_name = "./checkpoints/fgsm_random_8_255_smoothing_00125_seed_102_checkpoint_6125.pth"
checkpoint=torch.load(checkpoint_name)

target_model = init_model_metanode(checkpoint).eval()
target_model.to(device)
print()

In [18]:
source_solvers_solver_options_arr = get_solvers_solver_options_arr(source_solvers_config,
                                                                   source_model_ensemble_config,
                                                                   dtype, device)
target_solvers_solver_options_arr = get_solvers_solver_options_arr(target_solvers_config,
                                                                   target_model_ensemble_config,
                                                                   dtype, device)

print(f'Solvers for source model: {source_solvers_solver_options_arr} \n' +
      f'Solvers for target model: {target_solvers_solver_options_arr}')

Solvers for source model: [{'solvers': [<sopa.src.solvers.rk_parametric_order2stage2.RKOrder2Stage2 object at 0x1554bb8e2bd0>], 'solver_options': Namespace(ensemble_prob=None, ensemble_weights=None, solver_mode='standalone', switch_probs=None)}] 
Solvers for target model: [{'solvers': [<sopa.src.solvers.rk_parametric_order2stage2.RKOrder2Stage2 object at 0x1554bb8e2550>], 'solver_options': Namespace(ensemble_prob=None, ensemble_weights=None, solver_mode='standalone', switch_probs=None)}]


In [19]:
# FGSM attack
attack = fb.attacks.FGSM(random_start=False)

epsilons = [8/255.]

source_fmodel = fb.PyTorchModel(source_model, bounds=(0, 1), device=device, preprocessing=source_preprocessing)

In [20]:
from black_box import attack_validate

robust_accuracies = attack_validate(attack,
                                    epsilons,
                                    test_loader,
                                    source_model_ensemble_config,
                                    source_solvers_solver_options_arr,
                                    target_solvers_solver_options_arr,
                                    source_model=source_model,
                                    source_fmodel=source_fmodel,
                                    target_model=target_model,
                                    data_inv_normalize=data_inv_normalize,
                                    target_normalize=target_normalize,
                                    )

for eps, acc in zip(epsilons, robust_accuracies):
    print(f'eps: {int(eps*255)}, robust_accuracy: {acc}')

eps: 8, robust_accuracy: 0.41245993971824646
